---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    algo_list = list()
    for G in P1_Graphs:
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        degree_hist = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values]
        cluster = nx.average_clustering(G)
        
        if len(degree_hist)>10:
            algo_list.append('PA')
        elif cluster < 0.1:
            algo_list.append('SW_H')
        else:
            algo_list.append('SW_L')
    return algo_list

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [5]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [ ]:
def salary_predictions():
    
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import GridSearchCV

    #data preparation
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['degree_centrality'] = pd.Series(nx.degree_centrality(G))
    df['closeness'] = pd.Series(nx.closeness_centrality(G, normalized=True))
    df['betweeness'] = pd.Series(nx.betweenness_centrality(G, normalized=True))
    df_use = df[~pd.isnull(df['ManagementSalary'])]
    df_prediction = df[pd.isnull(df['ManagementSalary'])]

    #lr training
    X = df_use.drop('ManagementSalary', axis = 1)
    X_pred = df_prediction.drop('ManagementSalary', axis = 1)
    y = df_use['ManagementSalary']

    lr = LogisticRegression()

    grid_auc = GridSearchCV(lr, param_grid = ({'C': np.arange(1,10)}), scoring = 'roc_auc')
    grid_auc.fit(X, y)

    c = grid_auc.best_params_['C']
    score = grid_auc.best_score_

    #lr prediction
    lr = LogisticRegression(C=c).fit(X, y)
    pred_prob = lr.predict_proba(X_pred)[:, 1]
    results = pd.Series(pred_prob,X_pred.index)

    return results

In [ ]:
salary_predictions()

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import GridSearchCV

    future_connections['pref_attachment'] = [list(nx.preferential_attachment(G, [node_pair]))[0][2]
                                             for node_pair in future_connections.index]
    future_connections['comm_neighbors'] = [len(list(nx.common_neighbors(G, node_pair[0], node_pair[1]))) 
                                            for node_pair in future_connections.index]
    df_use = future_connections[~future_connections['Future Connection'].isnull()]
    df_prediction = future_connections[future_connections['Future Connection'].isnull()]

    #lr training
    X = df_use.drop('Future Connection', axis = 1)
    X_pred = df_prediction.drop('Future Connection', axis = 1)
    y = df_use['Future Connection']

    lr = LogisticRegression()

    grid_auc = GridSearchCV(lr, param_grid = ({'C': np.arange(1,10)}), scoring = 'roc_auc')
    grid_auc.fit(X, y)

    c = grid_auc.best_params_['C']
    score = grid_auc.best_score_

    #lr prediction
    lr = LogisticRegression(C=c).fit(X, y)
    pred_prob = lr.predict_proba(X_pred)[:, 1]
    results = pd.Series(pred_prob,X_pred.index)

    return results

In [ ]:
new_connections_predictions()

(107, 348)    0.038524
(542, 751)    0.014628
(20, 426)     0.641273
(50, 989)     0.014907
(942, 986)    0.015212
(324, 857)    0.014869
(13, 710)     0.164031
(19, 271)     0.108609
(319, 878)    0.015005
(659, 707)    0.014657
(49, 843)     0.015093
(208, 893)    0.014724
(377, 469)    0.006418
(405, 999)    0.026921
(129, 740)    0.022674
(292, 618)    0.023313
(239, 689)    0.015025
(359, 373)    0.009140
(53, 523)     0.040073
(276, 984)    0.015078
(202, 997)    0.015138
(604, 619)    0.049933
(270, 911)    0.015020
(261, 481)    0.080021
(200, 450)    0.856979
(213, 634)    0.014543
(644, 735)    0.049076
(346, 553)    0.013866
(521, 738)    0.012630
(422, 953)    0.022210
                ...   
(672, 848)    0.015020
(28, 127)     0.972472
(202, 661)    0.013754
(54, 195)     0.999926
(295, 864)    0.014825
(814, 936)    0.014585
(839, 874)    0.015212
(139, 843)    0.014849
(461, 544)    0.011815
(68, 487)     0.011677
(622, 932)    0.014758
(504, 936)    0.019084
(479, 528) 